In [1]:
import numpy as np
import scipy as sp
import scipy.stats
import struct
import requests
import cPickle
import random 
import matplotlib.pyplot as plt
import collections
import pickle
from sklearn.decomposition import PCA
import itertools
from sklearn.cluster import KMeans
from nltk.cluster import KMeansClusterer,cosine_distance
from nltk.cluster import KMeansClusterer, euclidean_distance
%matplotlib inline

In [2]:
import nltk
from nltk.corpus import brown

In [3]:
fv=brown.words()

In [4]:
len(fv)

1161192

In [5]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weiweili/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
#remove stopwords
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
rfv = [w for w in fv if w.lower() not in stopwords]


In [7]:
#remove punctuation
import re
import string
regex = re.compile('[%s]' % re.escape(string.punctuation))

new_rfv=[]
for token in rfv: 
    new_token = regex.sub(u'', token)
    if not new_token == u'':
        new_rfv.append(new_token)


In [10]:
new_rfv=[x.lower() for x in new_rfv]

In [13]:
fdist = nltk.FreqDist(w.lower() for w in new_rfv)
fdistV=fdist.most_common(5000)
fdistC=fdist.most_common(1000)
V=[word for (word, count) in fdistV]
C=[word for (word, count) in fdistC]


In [907]:

n=np.zeros([len(V),len(C)])
for i in range(len(V)):
    if i % 500 == 0:
        print i
    text_pos=[pos for pos, word in enumerate(new_rfv) if word==V[i]]
    for k in range(len(text_pos)):
        if (text_pos[k]>=2 and (text_pos[k]+2-len(new_rfv)+1)<=0):
            window=set([new_rfv[text_pos[k]-2],new_rfv[text_pos[k]-1],new_rfv[text_pos[k]+1],new_rfv[text_pos[k]+2]])
            #w=[word for word in C if word in window]
            C_pos=[pos for pos, word in enumerate(C) if word in window]
        elif(text_pos[k]>=2 and (text_pos[k]+2-len(new_rfv)+1)==1):
            window=set([new_rfv[text_pos[k]-2],new_rfv[text_pos[k]-1],new_rfv[text_pos[k]+1]])
            #w=[word for word in C if word in window]
            C_pos=[pos for pos, word in enumerate(C) if word in window]
        elif (text_pos[k]>=2 and (text_pos[k]+2-len(new_rfv)+1)==2):
            window=set([new_rfv[text_pos[k]-2],new_rfv[text_pos[k]-1]])
            #w=[word for word in C if word in window]
            C_pos=[pos for pos, word in enumerate(C) if word in window]
        elif (text_pos[k]==1):
            window=set([new_rfv[text_pos[k]-1],new_rfv[text_pos[k]+1],new_rfv[text_pos[k]+2]])
            #w=[word for word in C if word in window]
            C_pos=[pos for pos, word in enumerate(C) if word in window]
        elif (text_pos[k]==0):
            window=set([new_rfv[text_pos[k]+1],new_rfv[text_pos[k]+2]])
            #w=[word for word in C if word in window]
            C_pos=[pos for pos, word in enumerate(C) if word in window]
        n[i][C_pos]+=1

            

0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [304]:
with open('nwc_hw4.txt','wb') as f:
    pickle.dump(n,f)

In [14]:
with open('nwc_hw4.txt','rb') as f2:
    nwc=pickle.load(f2)
print(nwc.shape)

(5000, 1000)


In [15]:
nwc

array([[ 65.,  71.,  44., ...,   0.,   1.,   1.],
       [ 71.,  68.,  66., ...,   2.,   1.,   4.],
       [ 41.,  64.,  30., ...,   0.,   0.,   1.],
       ..., 
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  1.,   0.,   0., ...,   0.,   0.,   0.]])

In [26]:
Vcount=[count for (word, count) in fdistV]
ni=np.sum(nwc, axis=1) #5000*1
ind_zero=[pos for pos,n in enumerate(ni) if n==0]
ni[ind_zero]=1
Pcw=nwc/ni[:,None] #5000*1000
Pc=np.sum(nwc,axis=0)/nwc.sum() #1000*1

In [28]:
epsion = 1e-12
tmp=(Pcw+epsion)/(Pc+epsion)
tmp=np.log(tmp)
fcw=np.maximum(0,tmp)

In [29]:
##PCA
pca = PCA(n_components=100)
pca_fcw=pca.fit_transform(fcw)

In [213]:
##Nearest neighbor results
words=['communism','autumn','cigarette','pulmonary', 'mankind', 'africa', 'chicago', 'revolution', 'september',
'chemical', 'detergent', 'dictionary', 'storm', 'worship', 'head','brown','business','face','money','air',
      'month', 'third','seven','parents','happy']

ind_words=[pos for pos, word in enumerate(V) if word in words ]

tmp=[]
for i in range(len(words)):
    tmp.append(V[ind_words[i]])
words=tmp

In [214]:
dist=np.zeros([len(V),len(words)])
for i in range(len(words)):
    dist[:,i]=1-np.dot(pca_fcw,pca_fcw[ind_words[i],:])/(np.linalg.norm(pca_fcw,axis=1)* \
                                                        np.linalg.norm(pca_fcw[ind_words[i],:]))

In [215]:
ind_mindist=dist.argsort(axis=0)[1]
nnWords=[]
for i in range(len(ind_mindist)):
    nnWords.append(V[ind_mindist[i]])

In [219]:
print nnWords

[u'hands', u'local', u'eyes', u'pay', u'water', u'second', u'hair', u'last', u'ten', u'children', u'couldnt', u'kay', u'reflection', u'weapons', u'instances', u'text', u'kay', u'kay', u'nation', u'theological', u'artery', u'eighteenth', u'slid', u'fabrics', u'winter']


In [218]:
print words

[u'head', u'business', u'face', u'money', u'air', u'third', u'brown', u'month', u'seven', u'parents', u'happy', u'chicago', u'communism', u'revolution', u'chemical', u'dictionary', u'september', u'africa', u'mankind', u'worship', u'pulmonary', u'storm', u'cigarette', u'detergent', u'autumn']


In [196]:
## K-means clustering 
model = KMeans(n_clusters=100)
model.fit(pca_fcw)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=100, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [197]:
center=model.predict(pca_fcw)  #100  ##center for every word in V

In [198]:
ind_clusters=[[] for i in range(100)]
import itertools
for i in range(100):
    a=np.where(center==i)
    ind_clusters[i]=list(itertools.chain.from_iterable(a))


In [203]:
clusters=[[] for i in range(100)]
for i in range(100):
    clusters[i]=[word for pos, word in enumerate(V) if pos in ind_clusters[i]]

In [249]:
##nltk kmeans cosine_distance

from nltk.cluster import KMeansClusterer,cosine_distance
clusterer2=KMeansClusterer(100, distance=cosine_distance, 
                      repeats=5,avoid_empty_clusters=True)
clusters2 = clusterer2.cluster(pca_fcw, True)


In [140]:
print len(clusters2), min(clusters2),max(clusters2)

5000 0 99


In [248]:
##nltk kmeans euclidean_distance
from nltk.cluster import KMeansClusterer, euclidean_distance
clusterer = KMeansClusterer(100, euclidean_distance, repeats=5,avoid_empty_clusters=True)
clusters = clusterer.cluster(pca_fcw, True)  #center for every word in V

In [142]:
print len(clusters), min(clusters),max(clusters)

5000 0 99


In [404]:
#euclidean_distance:  index of every center in clusters, words in that cluster
ind_cluster=[pos for pos, center in enumerate(clusters) if center==2]
word_cluster=[word for pos, word in enumerate(V) if pos in ind_cluster]

In [405]:
len(word_cluster)

17

In [406]:
print word_cluster

[u'capacity', u'economy', u'initial', u'dramatic', u'machinery', u'critical', u'developing', u'advertising', u'mere', u'accomplished', u'varied', u'examine', u'extend', u'complicated', u'economical', u'urgent', u'expanded']


In [353]:
#cosine_distance:  index of every center in clusters, words in that cluster
ind_cluster2=[pos for pos, center in enumerate(clusters2) if center==29]
word_cluster2=[word for pos, word in enumerate(V) if pos in ind_cluster2]

In [354]:
len(word_cluster2)

31

In [355]:
print word_cluster2

[u'speak', u'theres', u'exactly', u'suppose', u'begin', u'believed', u'interior', u'capable', u'object', u'dream', u'save', u'minds', u'remarks', u'knowing', u'surely', u'sold', u'promise', u'odd', u'saved', u'happens', u'discover', u'definite', u'arent', u'accurate', u'missing', u'examine', u'suspect', u'happening', u'meaningful', u'settle', u'intimate']
